# 라이브러리 로드 및 기본 세팅

In [1]:
!pip install catboost

     |████████████████████████████████| 76.1 MB 1.2 MB/s 


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, classification_report, precision_recall_curve

# 데이콘 점수

In [4]:
# 자체 점수 : 
# 제출 점수(제출 날짜) : 

# 설정

In [5]:
# cat
# test_size = 0.1
# one_hot_max_size = 2
# random_state : 5(range(10, 60, 10))
# iterations = 1000
# early_stopping_rounds = 77
# 순서형 특성 수치형으로 지정(4개)
# 'person_attribute_a_a', 'person_attribute_b', 'person_prefer_e', 'contents_attribute_e'
# probability_threshold :

In [6]:
# 제거된 특성(7개)
# 'id',
# 'contents_open_dt',
# 'contents_rn',
# 'person_prefer_f',
# 'person_prefer_g',
# 'h_m_match_yn',
# 'h_s_match_yn'

# 데이터 불러오기

In [7]:
# train = pd.read_csv('../data/df_train.csv')
# test = pd.read_csv('../data/df_test.csv')
train = pd.read_csv('/content/drive/MyDrive/GitHub/Dacon/TP2(잡케어)/data/df_train.csv')
test = pd.read_csv('/content/drive/MyDrive/GitHub/Dacon/TP2(잡케어)/data/df_test.csv')

# 데이터 확인

In [8]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 501951 entries, 0 to 501950
Data columns (total 63 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   id                      501951 non-null  int64 
 1   d_l_match_yn            501951 non-null  bool  
 2   d_m_match_yn            501951 non-null  bool  
 3   d_s_match_yn            501951 non-null  bool  
 4   h_l_match_yn            501951 non-null  bool  
 5   h_m_match_yn            501951 non-null  bool  
 6   h_s_match_yn            501951 non-null  bool  
 7   person_attribute_a      501951 non-null  int64 
 8   person_attribute_a_1    501951 non-null  int64 
 9   person_attribute_b      501951 non-null  int64 
 10  person_prefer_c         501951 non-null  int64 
 11  person_prefer_d_1       501951 non-null  int64 
 12  person_prefer_d_2       501951 non-null  int64 
 13  person_prefer_d_3       501951 non-null  int64 
 14  person_prefer_e         501951 non-n

In [9]:
train.head()

,id,d_l_match_yn,d_m_match_yn,d_s_match_yn,h_l_match_yn,h_m_match_yn,h_s_match_yn,person_attribute_a,person_attribute_a_1,person_attribute_b,person_prefer_c,person_prefer_d_1,person_prefer_d_2,person_prefer_d_3,person_prefer_e,person_prefer_f,person_prefer_g,person_prefer_h_1,person_prefer_h_2,person_prefer_h_3,contents_attribute_i,contents_attribute_a,contents_attribute_j_1,contents_attribute_j,contents_attribute_c,contents_attribute_k,contents_attribute_l,contents_attribute_d,contents_attribute_m,contents_attribute_e,contents_attribute_h,person_rn,contents_rn,contents_open_dt,target,person_prefer_d_1_n,person_prefer_d_1_s,person_prefer_d_1_m,person_prefer_d_1_l,person_prefer_d_2_n,person_prefer_d_2_s,person_prefer_d_2_m,person_prefer_d_2_l,person_prefer_d_3_n,person_prefer_d_3_s,person_prefer_d_3_m,person_prefer_d_3_l,contents_attribute_d_n,contents_attribute_d_s,contents_attribute_d_m,contents_attribute_d_l,person_prefer_h_1_m,person_prefer_h_1_l,person_prefer_h_2_m,person_prefer_h_2_l,person_prefer_h_3_m,person_prefer_h_3_l,contents_attribute_h_m,contents_attribute_h_l,contents_attribute_l_n,contents_attribute_l_s,contents_attribute_l_m,contents_attribute_l_l
0,0,True,True,True,False,False,False,1,4,3,5,275,370,369,8,1,1,4,95,59,3,3,10,2,1,2,1608,275,1,4,139,618822,354805,2020-01-17 12:09:36,1,275,274,274,216,369,368,297,216,369,368,297,216,275,274,274,216,316,3,398,94,368,58,422,94,1607,1606,1605,2016
1,1,False,False,False,True,True,False,1,3,4,1,114,181,175,4,1,1,131,101,96,1,3,5,1,1,2,1608,275,1,4,133,571659,346213,2020-06-18 17:48:52,0,114,109,56,1,175,152,56,1,175,152,56,1,275,274,274,216,417,94,400,94,399,94,417,94,1607,1606,1605,2016
2,2,False,False,False,True,False,False,2,0,3,5,464,175,452,3,1,1,54,263,56,3,1,10,2,1,1,1600,94,1,4,53,399816,206408,2020-07-08 20:00:10,0,464,463,450,377,175,152,56,1,452,451,450,377,92,91,56,1,364,48,528,250,366,48,363,48,1599,1595,1572,2016
3,3,False,False,False,True,False,False,2,0,2,5,703,705,704,3,1,1,72,227,2,1,3,5,1,1,2,1608,275,5,3,74,827967,572323,2020-01-13 18:09:34,0,703,703,690,618,703,703,690,618,703,703,690,618,275,274,274,216,379,71,495,226,315,1,381,71,1607,1606,1605,2016
4,4,True,True,True,False,False,False,1,3,4,5,275,370,369,4,1,1,214,210,209,1,1,10,2,1,2,1608,275,1,4,74,831614,573899,2020-03-09 20:39:22,0,275,274,274,216,369,368,297,216,369,368,297,216,275,274,274,216,483,208,481,208,480,208,381,71,1607,1606,1605,2016


In [10]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46404 entries, 0 to 46403
Data columns (total 62 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   id                      46404 non-null  int64 
 1   d_l_match_yn            46404 non-null  bool  
 2   d_m_match_yn            46404 non-null  bool  
 3   d_s_match_yn            46404 non-null  bool  
 4   h_l_match_yn            46404 non-null  bool  
 5   h_m_match_yn            46404 non-null  bool  
 6   h_s_match_yn            46404 non-null  bool  
 7   person_attribute_a      46404 non-null  int64 
 8   person_attribute_a_1    46404 non-null  int64 
 9   person_attribute_b      46404 non-null  int64 
 10  person_prefer_c         46404 non-null  int64 
 11  person_prefer_d_1       46404 non-null  int64 
 12  person_prefer_d_2       46404 non-null  int64 
 13  person_prefer_d_3       46404 non-null  int64 
 14  person_prefer_e         46404 non-null  int64 
 15  pe

In [11]:
test.head()

,id,d_l_match_yn,d_m_match_yn,d_s_match_yn,h_l_match_yn,h_m_match_yn,h_s_match_yn,person_attribute_a,person_attribute_a_1,person_attribute_b,person_prefer_c,person_prefer_d_1,person_prefer_d_2,person_prefer_d_3,person_prefer_e,person_prefer_f,person_prefer_g,person_prefer_h_1,person_prefer_h_2,person_prefer_h_3,contents_attribute_i,contents_attribute_a,contents_attribute_j_1,contents_attribute_j,contents_attribute_c,contents_attribute_k,contents_attribute_l,contents_attribute_d,contents_attribute_m,contents_attribute_e,contents_attribute_h,person_rn,contents_rn,contents_open_dt,person_prefer_d_1_n,person_prefer_d_1_s,person_prefer_d_1_m,person_prefer_d_1_l,person_prefer_d_2_n,person_prefer_d_2_s,person_prefer_d_2_m,person_prefer_d_2_l,person_prefer_d_3_n,person_prefer_d_3_s,person_prefer_d_3_m,person_prefer_d_3_l,contents_attribute_d_n,contents_attribute_d_s,contents_attribute_d_m,contents_attribute_d_l,person_prefer_h_1_m,person_prefer_h_1_l,person_prefer_h_2_m,person_prefer_h_2_l,person_prefer_h_3_m,person_prefer_h_3_l,contents_attribute_h_m,contents_attribute_h_l,contents_attribute_l_n,contents_attribute_l_s,contents_attribute_l_m,contents_attribute_l_l
0,0,True,False,False,True,True,True,1,1,2,1,857,851,1227,4,1,1,263,56,49,1,3,10,2,1,2,1147,839,1,5,263,393790,236865,2020-12-01 02:24:18,854,853,824,744,847,847,824,744,1227,1227,1227,926,836,831,824,744,528,250,366,48,359,48,528,250,1146,1128,1021,2010
1,1,False,False,False,True,False,False,2,0,2,2,683,1086,662,2,1,1,258,263,49,1,3,5,1,1,2,1611,278,1,4,263,394058,236572,2020-12-17 05:42:53,683,677,659,618,1086,1086,1053,926,662,660,659,618,276,274,274,216,523,250,528,250,359,48,528,250,1610,1606,1605,2016
2,2,True,False,False,True,True,True,2,3,2,1,514,790,1233,0,1,1,177,170,171,3,3,3,1,1,2,1817,490,3,4,177,1002061,704612,2020-12-10 23:33:41,514,509,482,482,784,783,745,744,1227,1227,1227,926,490,490,482,482,453,169,451,169,452,169,453,169,1812,1811,1810,2020
3,3,True,False,False,True,True,True,1,2,2,5,114,181,175,4,1,1,177,170,171,3,3,5,1,1,2,101,150,5,3,177,1000813,704652,2020-12-03 19:44:55,114,109,56,1,175,152,56,1,175,152,56,1,145,123,56,1,453,169,451,169,452,169,453,169,101,100,99,2006
4,4,True,False,False,True,False,False,1,6,4,5,1082,1078,1056,5,1,1,178,177,4,3,1,5,1,1,1,985,1097,1,4,177,111146,704413,2020-12-11 21:24:34,1078,1078,1053,926,1078,1078,1053,926,1056,1054,1053,926,1097,1094,1093,926,454,169,453,169,316,3,453,169,984,980,954,2009


# 필요 데이터 추출

## 훈련에 의미가 적은 열 제거

In [12]:
# 제거된 특성(7개)
# 'id',
# 'contents_open_dt',
# 'contents_rn',
# 'person_prefer_f',
# 'person_prefer_g',
# 'h_m_match_yn',
# 'h_s_match_yn'

In [13]:
# 삭제할 특성
pop_col = ['id',
'contents_open_dt',
'contents_rn',
'person_prefer_f',
'person_prefer_g',
'h_m_match_yn',
'h_s_match_yn']
len(pop_col)

7

In [14]:
train = train.drop(pop_col, axis=1) 
test = test.drop(pop_col, axis=1)

In [15]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 501951 entries, 0 to 501950
Data columns (total 56 columns):
 #   Column                  Non-Null Count   Dtype
---  ------                  --------------   -----
 0   d_l_match_yn            501951 non-null  bool 
 1   d_m_match_yn            501951 non-null  bool 
 2   d_s_match_yn            501951 non-null  bool 
 3   h_l_match_yn            501951 non-null  bool 
 4   person_attribute_a      501951 non-null  int64
 5   person_attribute_a_1    501951 non-null  int64
 6   person_attribute_b      501951 non-null  int64
 7   person_prefer_c         501951 non-null  int64
 8   person_prefer_d_1       501951 non-null  int64
 9   person_prefer_d_2       501951 non-null  int64
 10  person_prefer_d_3       501951 non-null  int64
 11  person_prefer_e         501951 non-null  int64
 12  person_prefer_h_1       501951 non-null  int64
 13  person_prefer_h_2       501951 non-null  int64
 14  person_prefer_h_3       501951 non-null  int64
 15  

In [16]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46404 entries, 0 to 46403
Data columns (total 55 columns):
 #   Column                  Non-Null Count  Dtype
---  ------                  --------------  -----
 0   d_l_match_yn            46404 non-null  bool 
 1   d_m_match_yn            46404 non-null  bool 
 2   d_s_match_yn            46404 non-null  bool 
 3   h_l_match_yn            46404 non-null  bool 
 4   person_attribute_a      46404 non-null  int64
 5   person_attribute_a_1    46404 non-null  int64
 6   person_attribute_b      46404 non-null  int64
 7   person_prefer_c         46404 non-null  int64
 8   person_prefer_d_1       46404 non-null  int64
 9   person_prefer_d_2       46404 non-null  int64
 10  person_prefer_d_3       46404 non-null  int64
 11  person_prefer_e         46404 non-null  int64
 12  person_prefer_h_1       46404 non-null  int64
 13  person_prefer_h_2       46404 non-null  int64
 14  person_prefer_h_3       46404 non-null  int64
 15  contents_attribute_

## 카테고리형 특성 원핫인코딩시 컬럼 명 겹치는 문제 해결위해 특성명 변경

In [17]:
# 카테고리형 특성 원핫인코딩시 컬럼 명 겹치는 문제 해결위해 특성명 변경
train = train.rename(columns={'person_attribute_a_1':'person_attribute_a_a'})
train = train.rename(columns={'contents_attribute_j_1':'contents_attribute_j_a'})
train.head()

,d_l_match_yn,d_m_match_yn,d_s_match_yn,h_l_match_yn,person_attribute_a,person_attribute_a_a,person_attribute_b,person_prefer_c,person_prefer_d_1,person_prefer_d_2,person_prefer_d_3,person_prefer_e,person_prefer_h_1,person_prefer_h_2,person_prefer_h_3,contents_attribute_i,contents_attribute_a,contents_attribute_j_a,contents_attribute_j,contents_attribute_c,contents_attribute_k,contents_attribute_l,contents_attribute_d,contents_attribute_m,contents_attribute_e,contents_attribute_h,person_rn,target,person_prefer_d_1_n,person_prefer_d_1_s,person_prefer_d_1_m,person_prefer_d_1_l,person_prefer_d_2_n,person_prefer_d_2_s,person_prefer_d_2_m,person_prefer_d_2_l,person_prefer_d_3_n,person_prefer_d_3_s,person_prefer_d_3_m,person_prefer_d_3_l,contents_attribute_d_n,contents_attribute_d_s,contents_attribute_d_m,contents_attribute_d_l,person_prefer_h_1_m,person_prefer_h_1_l,person_prefer_h_2_m,person_prefer_h_2_l,person_prefer_h_3_m,person_prefer_h_3_l,contents_attribute_h_m,contents_attribute_h_l,contents_attribute_l_n,contents_attribute_l_s,contents_attribute_l_m,contents_attribute_l_l
0,True,True,True,False,1,4,3,5,275,370,369,8,4,95,59,3,3,10,2,1,2,1608,275,1,4,139,618822,1,275,274,274,216,369,368,297,216,369,368,297,216,275,274,274,216,316,3,398,94,368,58,422,94,1607,1606,1605,2016
1,False,False,False,True,1,3,4,1,114,181,175,4,131,101,96,1,3,5,1,1,2,1608,275,1,4,133,571659,0,114,109,56,1,175,152,56,1,175,152,56,1,275,274,274,216,417,94,400,94,399,94,417,94,1607,1606,1605,2016
2,False,False,False,True,2,0,3,5,464,175,452,3,54,263,56,3,1,10,2,1,1,1600,94,1,4,53,399816,0,464,463,450,377,175,152,56,1,452,451,450,377,92,91,56,1,364,48,528,250,366,48,363,48,1599,1595,1572,2016
3,False,False,False,True,2,0,2,5,703,705,704,3,72,227,2,1,3,5,1,1,2,1608,275,5,3,74,827967,0,703,703,690,618,703,703,690,618,703,703,690,618,275,274,274,216,379,71,495,226,315,1,381,71,1607,1606,1605,2016
4,True,True,True,False,1,3,4,5,275,370,369,4,214,210,209,1,1,10,2,1,2,1608,275,1,4,74,831614,0,275,274,274,216,369,368,297,216,369,368,297,216,275,274,274,216,483,208,481,208,480,208,381,71,1607,1606,1605,2016


In [18]:
# 카테고리형 특성 원핫인코딩시 컬럼 명 겹치는 문제 해결위해 특성명 변경
test = test.rename(columns={'person_attribute_a_1':'person_attribute_a_a'})
test = test.rename(columns={'contents_attribute_j_1':'contents_attribute_j_a'})
test.head()

,d_l_match_yn,d_m_match_yn,d_s_match_yn,h_l_match_yn,person_attribute_a,person_attribute_a_a,person_attribute_b,person_prefer_c,person_prefer_d_1,person_prefer_d_2,person_prefer_d_3,person_prefer_e,person_prefer_h_1,person_prefer_h_2,person_prefer_h_3,contents_attribute_i,contents_attribute_a,contents_attribute_j_a,contents_attribute_j,contents_attribute_c,contents_attribute_k,contents_attribute_l,contents_attribute_d,contents_attribute_m,contents_attribute_e,contents_attribute_h,person_rn,person_prefer_d_1_n,person_prefer_d_1_s,person_prefer_d_1_m,person_prefer_d_1_l,person_prefer_d_2_n,person_prefer_d_2_s,person_prefer_d_2_m,person_prefer_d_2_l,person_prefer_d_3_n,person_prefer_d_3_s,person_prefer_d_3_m,person_prefer_d_3_l,contents_attribute_d_n,contents_attribute_d_s,contents_attribute_d_m,contents_attribute_d_l,person_prefer_h_1_m,person_prefer_h_1_l,person_prefer_h_2_m,person_prefer_h_2_l,person_prefer_h_3_m,person_prefer_h_3_l,contents_attribute_h_m,contents_attribute_h_l,contents_attribute_l_n,contents_attribute_l_s,contents_attribute_l_m,contents_attribute_l_l
0,True,False,False,True,1,1,2,1,857,851,1227,4,263,56,49,1,3,10,2,1,2,1147,839,1,5,263,393790,854,853,824,744,847,847,824,744,1227,1227,1227,926,836,831,824,744,528,250,366,48,359,48,528,250,1146,1128,1021,2010
1,False,False,False,True,2,0,2,2,683,1086,662,2,258,263,49,1,3,5,1,1,2,1611,278,1,4,263,394058,683,677,659,618,1086,1086,1053,926,662,660,659,618,276,274,274,216,523,250,528,250,359,48,528,250,1610,1606,1605,2016
2,True,False,False,True,2,3,2,1,514,790,1233,0,177,170,171,3,3,3,1,1,2,1817,490,3,4,177,1002061,514,509,482,482,784,783,745,744,1227,1227,1227,926,490,490,482,482,453,169,451,169,452,169,453,169,1812,1811,1810,2020
3,True,False,False,True,1,2,2,5,114,181,175,4,177,170,171,3,3,5,1,1,2,101,150,5,3,177,1000813,114,109,56,1,175,152,56,1,175,152,56,1,145,123,56,1,453,169,451,169,452,169,453,169,101,100,99,2006
4,True,False,False,True,1,6,4,5,1082,1078,1056,5,178,177,4,3,1,5,1,1,1,985,1097,1,4,177,111146,1078,1078,1053,926,1078,1078,1053,926,1056,1054,1053,926,1097,1094,1093,926,454,169,453,169,316,3,453,169,984,980,954,2009


## 카테고리형 특성으로 지정할 특성

In [19]:
# 원본 데이터에서 순서형 특성
# 'person_attribute_a_a', 'person_attribute_b', 'person_prefer_e', 'contents_attribute_e'

In [20]:
# 순서형 특성 수치형으로 지정 : ⭕
# 'person_attribute_a_a', 'person_attribute_b', 'person_prefer_e', 'contents_attribute_e'

In [21]:
cat_f = train.columns.tolist()
numeric_cat = ['person_attribute_a_a', 'person_attribute_b', 'person_prefer_e', 'contents_attribute_e']
for col in numeric_cat:
  cat_f.remove(col)
cat_f.remove('target')

In [22]:
# 56 - 5 = 51개
len(cat_f)

51

# train, test 셋 준비

In [23]:
train.shape

(501951, 56)

In [24]:
X = train.drop(['target'], axis=1)
X.shape

(501951, 55)

In [25]:
y = train[['target']]
y.shape

(501951, 1)

# 모델 훈련/예측

In [28]:
all_thr_df = pd.DataFrame()
all_score_df = pd.DataFrame()
all_pred_proba_df = pd.DataFrame()

for test_size in [0.1, 0.2, 0.3, 0.4, 0.5]:
  score_df = pd.DataFrame(index=['thr_전', 'thr_후'])
  pred_proba_df = pd.DataFrame()
  thr_df = pd.DataFrame()
  for random_state in tqdm(range(10, 60, 10)):
    temp_score_list =[]
    # 데이터 셋 나누기
    X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=random_state, test_size=test_size,
                                                      shuffle=True, stratify=y)
    # 모델 생성
    cat = CatBoostClassifier(random_state=0, task_type='GPU', iterations=1000, early_stopping_rounds=77,
                          cat_features=cat_f, eval_metric='F1', one_hot_max_size=2)
    # 모델 훈련
    cat.fit(X_train, y_train, eval_set=(X_val, y_val), verbose=100)
    
    # 기본 평가
    val_pred = cat.predict(X_val)
    f1_score_raw = f1_score(y_val, val_pred)
    temp_score_list.append(f1_score_raw)

    # precision, recall, threshold 값 계산
    val_pred_proba = cat.predict_proba(X_val)
    precisions, recalls, thresholds = precision_recall_curve(y_val, val_pred_proba[:,1])
    threshold_boundary = thresholds.shape[0]

    # 최고 F1 값을 갖는 threshold 값 구하기
    pre = precisions[:threshold_boundary]
    re = recalls[:threshold_boundary]
    f1 = 2 * pre * re / (pre + re)
    optimal_idx = np.argmax(f1)
    optimal_threshold = thresholds[optimal_idx]
    thr_df[f'{random_state}'] = [optimal_threshold]

    # 구한 threshold값으로 제한해서 예측
    # 1의 예측값만 불러와도 되므로 [:,1]사용
    val_pred_th =  (val_pred_proba[:,1] >= optimal_threshold)
    val_pred_th = val_pred_th.astype(int)

    # threshold로 제한한 예측 평가
    f1_score_mod = f1_score(y_val, val_pred_th)
    temp_score_list.append(f1_score_mod)
    score_df[f'{random_state}'] = temp_score_list

    print("===============================================")
    print(f"test_size : {test_size}")
    print(f"random_state : {random_state}")
    print("threshold 적용 전 f1 score :", f1_score_raw)
    print("threshold 적용 후 f1 score :", f1_score_mod)
    print("===============================================")
    print()

    # test 셋 예측
    pred_proba = cat.predict_proba(test)
    pred_proba_df[f'{random_state}'] = pred_proba[:,1]
  
  #결과 종합
  all_thr_df = pd.concat([all_thr_df, thr_df])
  all_score_df = pd.concat([all_score_df, score_df])
  all_pred_proba_df = pd.concat([all_pred_proba_df, pred_proba_df], axis=1)

# about 42m

  0%|          | 0/5 [00:00<?, ?it/s]

Learning rate set to 0.04235
0:	learn: 0.6306710	test: 0.6331043	best: 0.6331043 (0)	total: 262ms	remaining: 4m 21s
100:	learn: 0.6499799	test: 0.6630939	best: 0.6633598 (98)	total: 16.2s	remaining: 2m 24s
200:	learn: 0.6602050	test: 0.6802622	best: 0.6802622 (200)	total: 32.4s	remaining: 2m 8s
300:	learn: 0.6654267	test: 0.6827868	best: 0.6827875 (266)	total: 48.5s	remaining: 1m 52s
400:	learn: 0.6695283	test: 0.6838343	best: 0.6840719 (393)	total: 1m 4s	remaining: 1m 36s
500:	learn: 0.6721068	test: 0.6859740	best: 0.6861131 (499)	total: 1m 20s	remaining: 1m 20s
bestTest = 0.6861694039
bestIteration = 518
Shrink model to first 519 iterations.
test_size : 0.1
random_state : 10
threshold 적용 전 f1 score : 0.686169403892944
threshold 적용 후 f1 score : 0.7153941302147755



 20%|██        | 1/5 [02:05<08:21, 125.43s/it]

Learning rate set to 0.04235
0:	learn: 0.6241250	test: 0.6281235	best: 0.6281235 (0)	total: 218ms	remaining: 3m 37s
100:	learn: 0.6515090	test: 0.6635505	best: 0.6635505 (100)	total: 16.4s	remaining: 2m 25s
200:	learn: 0.6615456	test: 0.6787594	best: 0.6787594 (200)	total: 32.7s	remaining: 2m 9s
300:	learn: 0.6667060	test: 0.6825448	best: 0.6826314 (294)	total: 48.5s	remaining: 1m 52s
400:	learn: 0.6700286	test: 0.6830176	best: 0.6832465 (383)	total: 1m 4s	remaining: 1m 36s
500:	learn: 0.6725884	test: 0.6839997	best: 0.6841794 (498)	total: 1m 20s	remaining: 1m 20s
600:	learn: 0.6747834	test: 0.6847947	best: 0.6849514 (577)	total: 1m 36s	remaining: 1m 3s
bestTest = 0.6849514192
bestIteration = 577
Shrink model to first 578 iterations.
test_size : 0.1
random_state : 20
threshold 적용 전 f1 score : 0.6849514191722134
threshold 적용 후 f1 score : 0.7167734251778415



 40%|████      | 2/5 [04:19<06:31, 130.66s/it]

...

 60%|██████    | 3/5 [04:11<02:45, 82.74s/it]

Learning rate set to 0.04452
0:	learn: 0.6108401	test: 0.6122576	best: 0.6122576 (0)	total: 148ms	remaining: 2m 27s
100:	learn: 0.6429592	test: 0.6565447	best: 0.6565447 (100)	total: 9.67s	remaining: 1m 26s
200:	learn: 0.6522812	test: 0.6654026	best: 0.6654026 (200)	total: 19.1s	remaining: 1m 15s
300:	learn: 0.6575434	test: 0.6686355	best: 0.6687842 (296)	total: 28.6s	remaining: 1m 6s
400:	learn: 0.6617227	test: 0.6699258	best: 0.6702560 (393)	total: 37.9s	remaining: 56.6s
500:	learn: 0.6647213	test: 0.6703586	best: 0.6703736 (464)	total: 47.1s	remaining: 46.9s
bestTest = 0.6703736317
bestIteration = 464
Shrink model to first 465 iterations.
test_size : 0.5
random_state : 40
threshold 적용 전 f1 score : 0.6703736317123901
threshold 적용 후 f1 score : 0.7043534967535433



 80%|████████  | 4/5 [05:40<01:25, 85.27s/it]

Learning rate set to 0.04452
0:	learn: 0.6355353	test: 0.6390227	best: 0.6390227 (0)	total: 150ms	remaining: 2m 29s
100:	learn: 0.6458144	test: 0.6600808	best: 0.6600808 (100)	total: 9.66s	remaining: 1m 25s
200:	learn: 0.6541194	test: 0.6684493	best: 0.6684493 (200)	total: 18.9s	remaining: 1m 15s
300:	learn: 0.6584461	test: 0.6707237	best: 0.6707682 (299)	total: 28.5s	remaining: 1m 6s
400:	learn: 0.6624548	test: 0.6710414	best: 0.6713576 (333)	total: 37.9s	remaining: 56.6s
500:	learn: 0.6644628	test: 0.6711945	best: 0.6715174 (448)	total: 47s	remaining: 46.8s
bestTest = 0.6715174125
bestIteration = 448
Shrink model to first 449 iterations.
test_size : 0.5
random_state : 50
threshold 적용 전 f1 score : 0.6715174124656865
threshold 적용 후 f1 score : 0.7040870604579561



100%|██████████| 5/5 [07:08<00:00, 85.73s/it]


## 결과 종합

In [34]:
all_score_df

,10,20,30,40,50
0,0.686169,0.684951,0.685376,0.684008,0.684365
1,0.715394,0.716773,0.713986,0.712423,0.713624
0,0.682093,0.683339,0.685463,0.680380,0.679918
1,0.711642,0.711525,0.712575,0.714002,0.709949
0,0.676592,0.679545,0.680247,0.679865,0.678811
1,0.709248,0.707529,0.707989,0.705970,0.710570
0,0.671439,0.674141,0.675320,0.675220,0.676527
1,0.703690,0.705403,0.704512,0.705572,0.707950
0,0.669509,0.671348,0.672234,0.670374,0.671517
1,0.705019,0.000000,0.702905,0.704353,0.704087


In [37]:
mod_all_score_df = all_score_df.copy()

In [39]:
mod_all_score_df.index= [['0.1','0.1','0.2','0.2','0.3','0.3','0.4','0.4','0.5','0.5'],
                        ['0','1','0','1','0','1','0','1','0','1']]

In [40]:
mod_all_score_df

10        20        30        40        50
0.1 0  0.686169  0.684951  0.685376  0.684008  0.684365
    1  0.715394  0.716773  0.713986  0.712423  0.713624
0.2 0  0.682093  0.683339  0.685463  0.680380  0.679918
    1  0.711642  0.711525  0.712575  0.714002  0.709949
0.3 0  0.676592  0.679545  0.680247  0.679865  0.678811
    1  0.709248  0.707529  0.707989  0.705970  0.710570
0.4 0  0.671439  0.674141  0.675320  0.675220  0.676527
    1  0.703690  0.705403  0.704512  0.705572  0.707950
0.5 0  0.669509  0.671348  0.672234  0.670374  0.671517
    1  0.705019  0.000000  0.702905  0.704353  0.704087

In [36]:
all_thr_df

,10,20,30,40,50
0,0.378409,0.377921,0.370538,0.382729,0.396402
0,0.385641,0.379673,0.391394,0.363218,0.375285
0,0.373730,0.384465,0.390848,0.384499,0.379938
0,0.372317,0.384494,0.373631,0.371829,0.362747
0,0.375448,0.893802,0.378210,0.382710,0.379275


### 오류난 특성 제거

In [ ]:
# del_col = ['30', '40', '60', '70']
# pred_proba_df = pred_proba_df.drop(del_col, axis=1)
# thr_df = thr_df.drop(del_col, axis=1)
# score_df = score_df.drop(del_col, axis=1)

In [ ]:
pred_proba_df['mean'] = pred_proba_df.mean(axis=1)
thr_df['mean'] = thr_df.mean(axis=1)
score_df['mean'] = score_df.mean(axis=1)

### threshold 값

In [ ]:
all_thr_df

,10,20,30,40,50,60,70,80,90,100,mean
0,0.391287,0.388262,0.373833,0.378376,0.375751,0.377859,0.371517,0.378718,0.36671,0.385416,0.378773


## 자체평가

In [ ]:
all_score_df

,10,20,30,40,50,60,70,80,90,100,mean
threshold 적용 전,0.680541,0.682001,0.680907,0.681353,0.680497,0.678795,0.675294,0.679191,0.679578,0.683618,0.680178
threshold 적용 후,0.710426,0.711445,0.709934,0.709564,0.711296,0.707484,0.709196,0.712188,0.710721,0.710469,0.710272


## test 예측

In [ ]:
pred = pred_proba_df['mean'].tolist() >= thr_df['mean'].values
pred = pred.astype(int)
pred

array([1, 0, 1, ..., 1, 1, 1])

In [ ]:
np.unique(pred, return_counts=True)

(array([0, 1]), array([10268, 36136]))

# 제출파일 불러오기

In [ ]:
# sub = pd.read_csv('../data/sample_submission.csv')
sub = pd.read_csv('/content/drive/MyDrive/GitHub/Dacon/TP2(잡케어)/data/sample_submission.csv')
sub.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46404 entries, 0 to 46403
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   id      46404 non-null  int64
 1   target  46404 non-null  int64
dtypes: int64(2)
memory usage: 725.2 KB


# 제출파일 생성

In [ ]:
sub['target'] = pred
sub['target'].value_counts()

1    36136
0    10268
Name: target, dtype: int64

In [ ]:
file_ver = '5.5'
# sub.to_csv(f'sub_ver{file_ver}.csv', index=False)
sub.to_csv(f'/content/drive/MyDrive/GitHub/Dacon/TP2(잡케어)/03_모델선정및개선/sub_ver{file_ver}.csv', index=False)

# 코드 마지막